In [1]:
%matplotlib inline
import pandas as pd
import tqdm
import numpy as np
from skimage.filters import threshold_otsu
%load_ext autoreload
%autoreload 2
import palm_utils as pu
from peaks.peakfinder import PeakFinder
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, PowerNorm
import glob

In [2]:
def palm_hist(df, yx_shape, subsampling=1):
    bins = [np.arange(s + subsampling, step=subsampling) - subsampling / 2 for s in yx_shape]
    # ungrouped 2d histogram to find peaks, ungrouped to make beads really stand out
    return np.histogramdd(df[["y0", "x0"]].values, bins=bins)[0]

In [3]:
data_dict = {path:pd.read_hdf(path, "dc") for path in glob.iglob("Localizations/*.h5")}

In [9]:
%time hists = {k:palm_hist(v, (512, 512), 0.1) for k, v in data_dict.items()}

C:\Users\hoffmand\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\lib\function_base.py:973: RuntimeWarning: invalid value encountered in greater_equal
  not_smaller_than_edge = (sample[:, i] >= edges[i][-1])


Wall time: 39.7 s


In [8]:
import skimage.external.tifffile as tif

In [11]:
for k, v in hists.items():
    tif.imsave(k.replace(".h5",".tif"), v.astype(np.uint32))

In [5]:
def palm_hist3d(df, yx_shape, zrange, subsampling=1, zstep=1):
    bins = [np.arange(s + subsampling, step=subsampling) - subsampling / 2 for s in yx_shape]
    bins = [np.arange(*zrange, zstep)] + bins
    # ungrouped 2d histogram to find peaks, ungrouped to make beads really stand out
    return np.histogramdd(df[["z0", "y0", "x0"]].values, bins=bins)[0]

In [6]:
hists3d = {k:palm_hist3d(v, (512, 512), (-20, 20), 0.1, 2) for k, v in data_dict.items()}

C:\Users\hoffmand\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\lib\function_base.py:973: RuntimeWarning: invalid value encountered in greater_equal
  not_smaller_than_edge = (sample[:, i] >= edges[i][-1])


In [12]:
for k, v in hists3d.items():
    if v.max() < 2**16:
        tif.imsave("E:\\Junk\\" + k.replace(".h5",".tif"), v.astype(np.uint16), resolution=(0.013, 0.013))
    else:
        tif.imsave("E:\\Junk\\" + k.replace(".h5",".tif"), v.astype(np.uint32), resolution=(0.013, 0.013))